# Project Report

**Summary**
Our proposal was to collect data for weather and traffic conditions in the Perth Metro region for the week of 25th to 31st of December 2017 using the 5 nearest weather stations.
Our original data source DPIRD turned out to be too convoluted to be able to extract the data so we used the Bureau of Meteorology data instead. We also decided to instead use the full year of 
2017 to get a bigger dataset for analysis.
This collated information could then be used for further analysis if required. 


**Data Sources**
Mainroads open data - "Historic Incidents"
BOM.wa.gov.au - "Climate data online"

**Analysis**
Mainroads data was formatted in a csv file which we then transformed and cleaned up in pandas, we reduced the columns from 18 down to 8. We needed to have a date column so that we could 
aggregate the weather data and find the nearest weather station to the incident, further analysis could then be carried out to see if the weather may have been a factor. 

**The columns we used for the transformed data are:**

| Date       | Time     | Type       | Conditions     | Lat        | Long       | Rainfall | Temperature |
|------------|----------|------------|----------------|------------|------------|----------|-------------|
| 2017-01-01 | 16:59:00 | Road Crash | All Lanes Open | -32.065714 | 115.894297 | 0.0      | 31.3        |


1. start of incident "date", this tells us when the incident is first reported. We needed to split this column as it had timestamp on it also - to be able to join to the weather data.  
&nbsp;
2. "time", this tells us the time of day the incident took place.  
&nbsp;
3. the incident "type", this tells us if it was a bushfire, crash, breakdown and so on.  
&nbsp;
4. Traffic "conditions", this tell us if it caused any lanes to be closed on the road the incident happened.  
&nbsp;
5. Latitude & Longitude, these two we kept in the data frame so that we could aggregate the weather data and find the closest weather station to the incident. However we did not end up using this column, but for further analyiss this could be used for mapping.  
&nbsp;
6. "Rainfall" in mm, shows how much rainfall on the day.  
&nbsp;
7. Maximum "Temperature", shows us what the temperature was on the day.  
&nbsp;

**Database**
used is sql lite as we ended up doing our data cleaning and merging in the notebook and did not have an overly complicated database.

In [1]:
# Dependancies
import pandas as pd
import datetime

#use this as no error in VScode
pd.options.mode.chained_assignment = None

# Import data sources   
traffic_data = "../Resources/Historic_Incidents_2016_2017.csv"
rain_data = "../Resources/IDCJAC0009_009225_2017_Rainfall.csv"
temp_data = "../Resources/IDCJAC0010_009225_2017_Temp.csv"

# Extraction

In [2]:
# Read in traffic data
data_df=pd.read_csv(traffic_data)

In [3]:
# import Temperature Data
temp_df = pd.read_csv(temp_data)

In [4]:
# Import and clean rainfall Data
rain_df = pd.read_csv(rain_data)

# Tranformation

In [5]:
# Filter to Metro only
data_df = data_df.loc[data_df["Operations_Centre"]=="Metro"]

In [6]:
# Remove other columns
new_data = data_df[["WST_Start","Incident_Type","TrafficCondition","Lat","Long"]]


In [7]:
# Filter to selected timerange (01/12 - 31/12)
date_df = new_data.loc[(new_data['WST_Start'] > "2017/01/01") & (new_data['WST_Start'] <= "2017/12/31")]

# Split Time of Incident into Date + Time
date_df[['Date','Time']] = date_df["WST_Start"].str.split(" ",expand=True)

# Convert Date to datetime object for merge
date_df["Date"] = date_df["Date"].str.replace('/','-')


In [8]:
# Reorder Columns for database
date_df = date_df[["Date","Time","Incident_Type","TrafficCondition","Lat","Long"]]

# Rename Columns for datebase
traffic_df = date_df.rename(columns={"Date":"date","Time":"time","Incident_Type":"type","TrafficCondition":"conditions","Lat":"lat","Long":"long"})

In [9]:
# remove columns
weather_df = rain_df[["Year", "Month", "Day", "Rainfall amount (millimetres)"]]

In [10]:
# Merge Rain and Temp into one df
weather_df["Maximum temperature (Degree C)"] = temp_df["Maximum temperature (Degree C)"]

In [11]:
# Combine Time columns into one Date object
weather_df["Date"] = pd.to_datetime(weather_df[['Year','Month','Day']])

In [12]:
# Reorder columns for database
weather_df = weather_df[["Date","Rainfall amount (millimetres)","Maximum temperature (Degree C)"]]

# Rename columns for database
weather_df = weather_df.rename(columns={"Date":"date","Rainfall amount (millimetres)":"rainfall","Maximum temperature (Degree C)":"temperature"})


In [13]:
#convert back to datetime string
weather_df['date'] = weather_df['date'].dt.date

In [14]:
#this is the merged rainfall table and temperature table
weather_df

,date,rainfall,temperature
0,2017-01-01,0.0,31.3
1,2017-01-02,0.0,37.1
2,2017-01-03,0.0,40.0
3,2017-01-04,0.0,40.9
4,2017-01-05,0.0,24.9
...,...,...,...
360,2017-12-27,0.0,25.2
361,2017-12-28,0.0,25.8
362,2017-12-29,0.0,28.3
363,2017-12-30,0.0,34.7


In [15]:
traffic_df

,date,time,type,conditions,lat,long
0,2017-01-01,16:59:00,Road Crash,All Lanes Open,-32.065714,115.894297
2,2017-01-02,04:50:59,Road Crash,All Lanes Open,-31.674911,115.711201
3,2017-01-02,07:30:00,Road Crash,All Lanes Open,-31.936552,115.838520
4,2017-01-02,08:16:59,Road Crash,All Lanes Open,-31.892157,116.015323
5,2017-01-02,12:13:00,Debris / Trees / Lost Loads,Right Centre Lane(s) Blocked,-31.936639,115.838076
...,...,...,...,...,...,...
7992,2017-12-30,20:03:59,Road Crash,Right Lane(s) Blocked,-31.949386,115.871415
7993,2017-12-30,23:23:00,Break Down / Tow Away,Right Emergency Lane Blocked,-31.927353,115.831052
11391,2017-01-06,14:35:00,Break Down / Tow Away,NaN,-31.949682,115.849312
11392,2017-01-06,15:24:59,Duplicate Entry,NaN,-31.940631,116.015960


# Loading

In [16]:
# Import SQL Alchemy
# Import and establish Base for which classes will be constructed
# Import modules to declare columns and column data types
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
Base=declarative_base()
from sqlalchemy import Column, Integer, String, Float

In [17]:
#create class for weather table
class weather(Base):
    __tablename__ = "weather"
    date = Column(String, primary_key=True)
    rainfall = Column(Float)
    temperature = Column(Float)

In [18]:
#create class for incidents table
class incidents(Base):
    __tablename__="incidents"
    date = Column(String, primary_key=True)
    time = Column(String)
    type = Column(String)
    conditions = Column(String)
    lat = Column(Float)
    long = Column(Float)

In [19]:
engine = create_engine("sqlite:///ETL.sqlite")
conn = engine.connect()

In [20]:
weather_df.to_sql('weather',con=conn,if_exists='replace')

In [21]:
traffic_df.to_sql('incidents',con=conn,if_exists='replace')

In [22]:
engine.execute("SELECT * FROM incidents").fetchall()

[(0, '2017-01-01', '16:59:00', 'Road Crash', 'All Lanes Open', -32.0657141, 115.8942969),
 (2, '2017-01-02', '04:50:59', 'Road Crash', 'All Lanes Open', -31.67491075, 115.71120069999999),
 (3, '2017-01-02', '07:30:00', 'Road Crash', 'All Lanes Open', -31.93655151, 115.83852009999998),
 (4, '2017-01-02', '08:16:59', 'Road Crash', 'All Lanes Open', -31.89215734, 116.01532279999999),
 (5, '2017-01-02', '12:13:00', 'Debris / Trees / Lost Loads', 'Right Centre Lane(s) Blocked', -31.9366389099999, 115.838076),
 (6, '2017-01-02', '13:04:00', 'Break Down / Tow Away', 'Left Lane(s) Blocked', -31.97080762, 115.84707420000001),
 (7, '2017-01-02', '18:56:59', 'Break Down / Tow Away', 'All Lanes Open', -32.09230819, 115.8494439),
 (8, '2017-01-02', '19:31:00', 'Break Down / Tow Away', 'All Lanes Open', -32.09291554, 115.84913059999998),
 (9, '2017-01-03', '10:47:00', 'Debris / Trees / Lost Loads', 'All Lanes Open', -31.8824606699999, 115.8013439),
 (10, '2017-01-03', '14:35:59', 'Break Down / Tow A